In [4]:
import pandas as pd    
import numpy as np 

import pyecharts.options as opts
from pyecharts.globals import ThemeType
from pyecharts.charts import Line,Bar,Map,Pie,Grid,Tab,Timeline
 
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB 

df_comfirmed = pd.read_csv('data/world_comfirmed.csv',
                           header=0, index_col=[1, 0]).drop(['Lat', 'Long'], axis=1)
df_death = pd.read_csv('data/world_death.csv',
                       header=0, index_col=[1, 0]).drop(['Lat', 'Long'], axis=1)
df_recovery = pd.read_csv('data/world_recovery.csv',
                          header=0, index_col=[1, 0]).drop(['Lat', 'Long'], axis=1)

sum_comfirmed = df_comfirmed.groupby('Country/Region').sum()
sum_death = df_death.groupby('Country/Region').sum().iloc[:,-1]
sum_recovery = df_recovery.groupby('Country/Region').sum().iloc[:,-1]
 

In [15]:
name_map = {
    "US": "United States",
    "Korea, South":"Korea",
    "Congo (Brazzaville)":"Congo",
    "Congo (Kinshasa)":"Dem. Rep. Congo",
    "Czechia":"Czech Rep.",
    "North Macedonia":"Macedonia",
    "Bosnia and Herzegovina":"Bosnia and Herz."
}


def get_nation_name(name):
    if name in name_map:
        return name_map[name]
    return name

visual_opts = opts.VisualMapOpts(max_=100000, is_piecewise=True,
                                 pieces=[
                                     {"max": 100000, "min": 10001,
                                      "label": ">1000", "color": "#6A0C0C"},
                                     {"max": 10000, "min": 1001,
                                      "label": ">1000", "color": "#8A0808"},
                                     {"max": 1000, "min": 500,
                                      "label": "500-1000", "color": "#FF0000"},
                                     {"max": 499, "min": 100,
                                      "label": "100-499", "color": "#B8860B"},
                                     {"max": 99, "min": 10,
                                      "label": "10-99", "color": "#EEB422"},
                                     {"max": 9, "min": 1, "label": "1-9",
                                      "color": "#EEDD82"},
                                     {"max": 0, "min": 0, "label": "0",
                                      "color": "#FFFFFF"},
                                 ])
 
tl = Timeline()
for day in sum_comfirmed.columns:  
    s = sum_comfirmed[day]
    pairlst = [(get_nation_name( s.index[i]),int(s.values[i])) for i in range(len(s))]
    map0 = (
        Map(init_opts=opts.InitOpts(theme=ThemeType.DARK))
        .add("comfirmed case",pairlst, "world",is_map_symbol_show=False,label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(visualmap_opts=visual_opts) 
    )
    tl.add(map0, day) 

tl.load_javascript()

In [16]:
tl.render_notebook()